In [ ]:
using Plots
using LinearAlgebra
using DifferentialEquations
plotly()

In [ ]:
include("../src/SailorMoon.jl")   # relative path to main file of module
include("../../julia-r3bp/R3BP/src/R3BP.jl")

In [ ]:
param3b = SailorMoon.dyanmics_parameters()

In [ ]:
τ = 0.1
tmax = 0.4  # N = kg m / s^2
mdot = tmax / 9.8 / 3500  # kg/s -> Isp = 3500s

# # normalize
mstar = 5.972e24 / param3b.mu1 
tmax = 1e-2  #tmax / mstar / param3b.lstar * (param3b.tstar)^2 
mdot = 1e-3  #mdot / mstar * param3b.tstar 
mf = 1  #3000 / mstar  

n = 1
SCParam = [mdot, tmax, mf]

In [ ]:
param3b.mus

In [ ]:
# set up LPOArr 

lp = 2
Az_km = 28000.0
println("Halo guess Az_km: $Az_km")
northsouth = 3   # 1 or 3
guess0 = R3BP.halo_analytical_construct(param3b.mu2, lp, Az_km, param3b.lstar, northsouth)
res = R3BP.ssdc_periodic_xzplane([param3b.mu2,], guess0.x0, guess0.period, fix="z")
res.flag

x0_stm = vcat(res.x0, reshape(I(6), (6^2,)))[:]
prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, res.period, (param3b.mu2))
sol = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12)#, saveat=LinRange(0, period, n+1))
monodromy = R3BP.get_stm(sol, 6)   # get monodromy matrix

ys0 = R3BP.get_eigenvector(monodromy, true, 1)
# arrival LPO object
LPOArrival = SailorMoon.CR3BPLPO(
    res.x0, res.period, ys0, prob_cr3bp_stm, 1e-6, Tsit5(), 1e-12, 1e-12
);

In [ ]:
# extract optimization var
ϕ = 0.0
θm = deg2rad(120)

tof      = (90 * 86400) / param3b.tstar
c_launch = [mf*1.2, 3.0, 0.0, 0.0]  # m0, dv1, long, lat
c_arr    = [ϕ, θm]
tau1     = [1, 0.0, 0.0]  # discretization numbers are the same for the first & second arc
tau2     = [1, 0.0, 0.0]

# make a decision vector x 
x = []
append!(x,tof)
append!(x,c_launch)
append!(x,c_arr)
append!(x,tau1)
append!(x,tau2)
println(x)

In [ ]:
param3b.lstar/param3b.tstar

In [ ]:
# set up the default ODE
state0 = zeros(7)
tspan = zeros(2)
params = zeros(10)

Propagator = SailorMoon.ODEPropagator(
    Tsit5(), 1e-12, 1e-12, ODEProblem(SailorMoon.rhs_bcr4bp_with_mass!, state0, tspan, params)
);

In [ ]:
# propagate
res, state_fwd, state_bkwd, sol_fwd, sol_bkwd = SailorMoon.sf_propagate(x, n, SCParam, Propagator, param3b, LPOArrival);
print(res)

In [ ]:
Array(sol_fwd[1])

In [ ]:
Array(sol_bkwd[1])

In [ ]:
thetas = LinRange(0, 2π, 100)
xs_earth_circle = [-param3b.mu2*cos(theta) for theta in thetas];
ys_earth_circle = [-param3b.mu2*sin(theta) for theta in thetas];
xs_moon_circle = [(1-param3b.mu2)*cos(theta) for theta in thetas];
ys_moon_circle = [(1-param3b.mu2)*sin(theta) for theta in thetas];

In [ ]:
ptraj = plot(frame_style=:box, size=(700,700), aspect_ratio=:equal)
plot!(ptraj, Array(sol_fwd[1])[1,:], Array(sol_fwd[1])[2,:])
plot!(ptraj, Array(sol_bkwd[1])[1,:], Array(sol_bkwd[1])[2,:])

plot!(ptraj, xs_moon_circle, ys_moon_circle, color=:black, linestyle=:solid, label="Moon trajectory", linewidth=0.4)

ptraj